# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy
#import openweathermapy

In [2]:
# Output File (CSV)
output_data_file = "C:/Users/jmanj/OneDrive/Desktop/Resources/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [3]:
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=lat_range[0], high=lat_range[1], size=1500)
lngs = np.random.uniform(low=lng_range[0], high=lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city by passing lat and lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # append unique city to the list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print("There are {} cities found from random geo-coordinates.".format(len(cities)))

There are 631 cities found from random geo-coordinates.


# Create a set of random lat and lng combinations

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
# Identify nearest city for each lat, lng combination

for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

606

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
unit = "Imperial"

#api_key = "enter api key to execute"

base_url = f"http://api.openweathermap.org/data/2.5/weather?"
param = {"appid" : weather_api_key,
         "units" : unit}

# Create DataFrame with below parameters
city_weather_df = pd.DataFrame(columns=["City",
                                        "Cloudiness(%)",
                                        "Country",
                                        "Date",
                                        "Humidity(%)",
                                        "Lat",
                                        "Lng", 
                                        "Max Temp(F)",
                                        "Wind Speed(mph)"])

# get local current time
def current_date():
    localtime = time.asctime( time.localtime(time.time()) )
    return localtime.split(' ')[1]+"/"+localtime.split(' ')[2]+"/"+localtime.split(' ')[4]


print("Beginning Data Retrieval")
print("-----------------------------------")


# store each city weather information on DataFrame
i = 0
not_found = 0
for city in cities:
    param["q"] = city
    
    # Perform a weather check on each of the cities using a series of successive API calls    
    data = requests.get(base_url, params=param).json()
    try:
        city_weather_df.loc[i, "City"] = data["name"]
        city_weather_df.loc[i, "Cloudiness(%)"] = data["clouds"]["all"]
        city_weather_df.loc[i, "Country"] = data["sys"]["country"]
        city_weather_df.loc[i, "Date"] = current_date()
        city_weather_df.loc[i, "Humidity(%)"] = data["main"]["humidity"]
        city_weather_df.loc[i, "Lat"] = data["coord"]["lat"]
        city_weather_df.loc[i, "Lng"] = data["coord"]["lon"]
        city_weather_df.loc[i, "Max Temp(F)"] = data["main"]["temp_max"]
        city_weather_df.loc[i, "Wind Speed(mph)"] = data["wind"]["speed"]
        
        i += 1
        # Include a print log of each city as it's being processed with the city number and city name.
        print("Processing Record {:3d} of set {} | {} ".format(i, len(cities), data['name']))
              
    except KeyError:
        print("(Total {} cities out of {} not found...) ".format(not_found + 1, len(cities)))
        not_found += 1

print(f"""
-------------------------------------------
Data Retrieval Completed
-------------------------------------------""")

Beginning Data Retrieval
-----------------------------------


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

# Save CSV of all data retrieved
city_weather_df.to_csv(output_data_file)
city_weather_df.count()

In [ ]:
#Display dataframe
city_weather_df.head()

### Plotting the Data

In [ ]:
font = 12
lat = city_weather_df['Lat']
max_temp = city_weather_df['Max Temp(F)']
date = city_weather_df['Date'][0]

# Print analysis
print ("Minimum Temprature is {:.2f}" .format(max_temp.min()))
print ("Average Temprature is {:.2f}" .format(max_temp.mean()))
print ("Maximun Temprature is {:.2f}" .format (max_temp.max()))  
print ("Their are {} cities temprature are greater than 100 degree (F) \n" .format (city_weather_df[city_weather_df['Max Temp(F)']> 100].count()['City']))
 


plt.grid(True)
# create scatter plot for city latitude with max temprature
plt.scatter(lat, max_temp, marker="o", facecolors="lightblue", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Max Temperature (F)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Max Temparature ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/max_temp.png")
plt.show()

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

In [ ]:
font = 12
lat = city_weather_df['Lat']
humidity = city_weather_df['Humidity(%)']
date = city_weather_df['Date'][1]

# Print analysis
print ("Maximum Humidity {:.2f} %".format(humidity.max()))
print ("Minimum Humidity {:.2f} %".format(humidity.min()))
print ("Average Humidity {:.2f} %".format(humidity.mean()))

plt.grid(True)
# create scatter plot for humidity by city latitude
plt.scatter(lat, humidity, marker="o", facecolors="lightblue", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Humidity (%)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Humidity (%) ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/lat_humidity.png")
plt.show()

#### Latitude vs. Cloudiness Plot

In [ ]:
font = 12
lat = city_weather_df['Lat']
cloudiness = city_weather_df['Cloudiness(%)']
date = city_weather_df['Date'][1]

plt.grid(True)
# create scatter plot for cloudiness by city latitude
plt.scatter(lat, cloudiness, marker="o", facecolors="yellow", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Cloudiness (%)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Cloudiness (%) ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/lat_cloudiness.png")
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
font = 12
lat = city_weather_df['Lat']
wind_speed = city_weather_df['Wind Speed(mph)']
date = city_weather_df['Date'][1]

plt.grid(True)
# create scatter plot for wind speed mph by city latitude
plt.scatter(lat, wind_speed, marker="o", facecolors="red", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Wind Speed (mph)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Wind Speed ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/lat_wind_speed.png")
plt.show()

unit = "Imperial"
base_url = f"http://api.openweathermap.org/data/2.5/weather?"
param = {"appid":api_key,
        "units":unit}
city_weatherdf = pd.DataFrame(columns=["City",
                                       "Cloudiness(%)",
                                       "Country",
                                       "Date",
                                       "Humidity(%)",
                                       "Lat",
                                       "Lng",
                                       "Max Temp(F)",
                                       "Wind Speed(mph)"])
def presentdate():
    localtime = time.asctime( time.localtime(time.time()) )
    return localtime.split(' ')[1]+"/"+localtime.split(' ')[2]+"/"+localtime.split(' ')[4]

print("Data Retrieval Starting")
print("---------------------------------")

x = 0
notfound = 0
for city in cities:
    param["q"] = city
    data = requests.get(base_url, params=param).json()
    try:
        city_weatherdf.loc[i,"City"] = data["name"]
        city_weatherdf.loc[i, "Cloudiness(%)"] = data["clouds"]["all"]
        city_weatherdf.loc[i, "Country"] = data["sys"]["country"]
        city_weatherdf.loc[i,"Date"] = currentdate()
        city_weatherdf.loc[i,"Humidity(%)"] = data["main"]["humidity"]
        city_weatherdf.loc[i,"Lat"] = data["coord"]["lat"]
        city_weatherdf.loc[i,"Lng"] = data["coord"]["lon"]
        city_weatherdf.loc[i, "Max Temp(F)"] = data["main"]["temp_max"]
        city_weatherdf.loc[i, "Wind Speed(mph)"] = data["wind"]["speed"]
    
        x = x + 1
        
        print("Processing Record {:3d} of set {} | {} ".format(i, len(cities), data['name']))
              

    except KeyError: 
        print("(Total {} citites out of {} not found...) ".format(notfound+1, len(cities)))
        notfound = notfound + 1

print(f"""
-------------------------------------------
Data Retrieval Completed
-------------------------------------------""")


font = 12
lat = city_weather_df['Lat']
humidity = city_weather_df['Humidity(%)']
date = city_weather_df['Date'][1]

# Print analysis
print ("Maximum Humidity {:.2f} %".format(humidity.max()))
print ("Minimum Humidity {:.2f} %".format(humidity.min()))
print ("Average Humidity {:.2f} %".format(humidity.mean()))

plt.grid(True)
# create scatter plot for humidity by city latitude
plt.scatter(lat, humidity, marker="o", facecolors="lightblue", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Humidity (%)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Humidity (%) ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/lat_humidity.png")
plt.show()

font = 12
lat = city_weather_df['Lat']
wind_speed = city_weather_df['Wind Speed(mph)']
date = city_weather_df['Date'][1]

plt.grid(True)
# create scatter plot for wind speed mph by city latitude
plt.scatter(lat, wind_speed, marker="o", facecolors="red", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Wind Speed (mph)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Wind Speed ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/lat_wind_speed.png")
plt.show()

font = 12
lat = city_weather_df['Lat']
max_temp = city_weather_df['Max Temp(F)']
date = city_weather_df['Date'][0]

# Print analysis
print ("Minimum Temprature is {:.2f}" .format(max_temp.min()))
print ("Average Temprature is {:.2f}" .format(max_temp.mean()))
print ("Maximun Temprature is {:.2f}" .format (max_temp.max()))  
print ("Their are {} cities temprature are greater than 100 degree (F) \n" .format (city_weather_df[city_weather_df['Max Temp(F)']> 100].count()['City']))
 


plt.grid(True)
# create scatter plot for city latitude with max temprature
plt.scatter(lat, max_temp, marker="o", facecolors="lightblue", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Max Temperature (F)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Max Temparature ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/max_temp.png")
plt.show()

font = 12
lat = city_weather_df['Lat']
cloudiness = city_weather_df['Cloudiness(%)']
date = city_weather_df['Date'][1]

plt.grid(True)
# create scatter plot for cloudiness by city latitude
plt.scatter(lat, cloudiness, marker="o", facecolors="yellow", edgecolors="black")
# set x axis label
plt.xlabel("Latitude", fontsize = font)
# set y axis label
plt.ylabel("Cloudiness (%)", fontsize = font)
# set chart title 
plt.title(f"City Latitude vs. Cloudiness (%) ({date})\n", fontsize = font)
#  save png images to output folder
plt.savefig("output_data/lat_cloudiness.png")
plt.show()